In [1]:
from sklearn.cluster import AgglomerativeClustering
from _jsonnet import evaluate_file as jsonnet_evaluate_file
from transformers import AutoTokenizer, EncoderDecoderModel, logging, AutoModel
from models.bottleneck_encoder_decoder import BottleneckEncoderDecoderModel

In [2]:
from evaluation.clustering_utils import get_gold_markup, get_data_to_cluster, doc2vec_bert, calc_clustering_metrics

In [3]:
def get_text_to_vector_func(text_to_vec_func, model, tokenizer):
    if text_to_vec_func == 'bert-MeanSum':
        return lambda doc: doc2vec_bert(doc, model, tokenizer, 'MeanSum')
    elif text_to_vec_func == 'bert-FirstCLS':
        return lambda doc: doc2vec_bert(doc, model, tokenizer, 'FirstCLS')
    else:
        raise NotImplementedError

In [4]:
logging.set_verbosity_info()

In [5]:
from transformers import AutoTokenizer, AutoModel
  
model = AutoModel.from_pretrained("IlyaGusev/gen_title_tg_bottleneck_encoder", cache_dir='/data/aobuhtijarov/cache_trans')

https://huggingface.co/IlyaGusev/gen_title_tg_bottleneck_encoder/resolve/main/config.json not found in cache or force_download set to True, downloading to /data/aobuhtijarov/cache_trans/tmp1ozwje48


storing https://huggingface.co/IlyaGusev/gen_title_tg_bottleneck_encoder/resolve/main/config.json in cache at /data/aobuhtijarov/cache_trans/991124d78a9d7cbd22e8b0826146105684df44bb2ae45b55168b4680624bc0ac.87841bc3a554ba52cee53cb7afebef5b52689b040e8f3862468adb761234a540
creating metadata file for /data/aobuhtijarov/cache_trans/991124d78a9d7cbd22e8b0826146105684df44bb2ae45b55168b4680624bc0ac.87841bc3a554ba52cee53cb7afebef5b52689b040e8f3862468adb761234a540
loading configuration file https://huggingface.co/IlyaGusev/gen_title_tg_bottleneck_encoder/resolve/main/config.json from cache at /data/aobuhtijarov/cache_trans/991124d78a9d7cbd22e8b0826146105684df44bb2ae45b55168b4680624bc0ac.87841bc3a554ba52cee53cb7afebef5b52689b040e8f3862468adb761234a540
Model config BertConfig {
  "_name_or_path": "models/rubert",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_p

https://huggingface.co/IlyaGusev/gen_title_tg_bottleneck_encoder/resolve/main/pytorch_model.bin not found in cache or force_download set to True, downloading to /data/aobuhtijarov/cache_trans/tmpgexh0id1


storing https://huggingface.co/IlyaGusev/gen_title_tg_bottleneck_encoder/resolve/main/pytorch_model.bin in cache at /data/aobuhtijarov/cache_trans/b5b81398e41ba89668795bb85714d36bd544706c0d81dcb156bc174f71a7796e.f5e27d4e7fe6b73df075ea388c6e299caf1fb05d849335f1e7160326c7b5c682
creating metadata file for /data/aobuhtijarov/cache_trans/b5b81398e41ba89668795bb85714d36bd544706c0d81dcb156bc174f71a7796e.f5e27d4e7fe6b73df075ea388c6e299caf1fb05d849335f1e7160326c7b5c682
loading weights file https://huggingface.co/IlyaGusev/gen_title_tg_bottleneck_encoder/resolve/main/pytorch_model.bin from cache at /data/aobuhtijarov/cache_trans/b5b81398e41ba89668795bb85714d36bd544706c0d81dcb156bc174f71a7796e.f5e27d4e7fe6b73df075ea388c6e299caf1fb05d849335f1e7160326c7b5c682


OSError: Unable to load weights from pytorch checkpoint file for 'IlyaGusev/gen_title_tg_bottleneck_encoder' at '/data/aobuhtijarov/cache_trans/b5b81398e41ba89668795bb85714d36bd544706c0d81dcb156bc174f71a7796e.f5e27d4e7fe6b73df075ea388c6e299caf1fb05d849335f1e7160326c7b5c682'If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True. 

In [9]:
model = BottleneckEncoderDecoderModel.from_pretrained('IlyaGusev/gen_title_tg_bottleneck', cache_dir='/data/aobuhtijarov/cache_trans')

loading configuration file https://huggingface.co/IlyaGusev/gen_title_tg_bottleneck/resolve/main/config.json from cache at /data/aobuhtijarov/cache_trans/b69815ec1afbafe51db26fe35235e17d0a6b1637b729d7ead485eb65272cf172.339b64342cafb807be8ba7e2c3c5517f97dee1f06d5eb5d39639bca5324966b4
Model config EncoderDecoderConfig {
  "_name_or_path": "models/gen_title",
  "architectures": [
    "BottleneckEncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "models/rubert",
    "add_cross_attention": true,
    "architectures": null,
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "decoder_start_token_id": null,
    "directionality": "bidi",
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "eos_token_id": null,
    "finetuning_task": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label":

OSError: Unable to load weights from pytorch checkpoint file for 'IlyaGusev/gen_title_tg_bottleneck' at '/data/aobuhtijarov/cache_trans/90a823a53956d3e5bb62e88073be6a368b0d12887b8b65ea2c0f3a6e5a46b161.44bcad3a2d7e657e2170e7b929c77bf23bed7e28ee7f8a98f3f1de6a6d8b1611'If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True. 

In [ ]:
gold_markup_file = '/data/aobuhtijarov/datasets/telegram_news/ru_pairs_raw_markup.tsv'
clustering_data_file = '/data/aobuhtijarov/datasets/telegram_news/ru_clustering_data.jsonl'

In [6]:
### BEWARE

tokenizer = AutoTokenizer.from_pretrained('/data/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/', do_lower_case=False)

loading configuration file /data/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.2.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

Model name '/data/aobuhtijarov/models/rubert_cased_L-12_H-768_A-12_pt/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, be

In [16]:
max_tokens_text=196
text_to_vec_func='bert-FirstCLS'

In [17]:
gold_markup = get_gold_markup(gold_markup_file)
url2record, filename2url = get_data_to_cluster(clustering_data_file)
setattr(tokenizer, 'max_tokens_text', max_tokens_text)
text_to_vector_func = get_text_to_vector_func(text_to_vec_func, model, tokenizer)

print('Calculating embeddings...')
embeds = np.zeros((len(url2record.items()), 768))

for i, (url, record) in tqdm.tqdm(enumerate(url2record.items())):
    text = record["title"] + ' ' + record["text"]
    text = text.lower().replace('\xa0', ' ')
    embeds[i] = text_to_vector_func(text).detach().numpy().ravel()

print('Embeds shape =', embeds.shape)
assert len(embeds) == len(url2record.items())


NameError: name 'model' is not defined